In [1]:
# Main import block
import numpy as np
import pandas as pd
import requests
import cfscrape 
from bs4 import BeautifulSoup
import datetime
from datetime import date, datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import re
import asyncio


import time
from tqdm import tqdm

# Upload to google sheets
import gspread
#import df2gspread as d2g
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

# Upload to yandex
import yadisk

import signal
from contextlib import contextmanager

import warnings
warnings.filterwarnings('ignore')


In [220]:
# Стандартная функция
def kitko_parser():
    def kitco_raw():
        try:
                url = "https://www.kitco.com/price/fixes/london-fix"

                service = Service()
                driver = webdriver.Chrome(service=service)

                # driver.maximize_window()
                driver.get(url=url)

                time.sleep(2)

                driver.execute_script(
                    "window.scrollTo(0, window.scrollY + window.innerHeight);"
                )
                driver.execute_script(
                    "window.scrollTo(0, window.scrollY + window.innerHeight);"
                )

                time.sleep(2)

                html_code = driver.page_source

                return html_code

        except Exception as error:
            print(error)

        finally:
            driver.close()
            driver.quit()
    
    # Работа над полученными данными
    html_code = kitco_raw()
    soup = BeautifulSoup(html_code)
    
    # Выделим данные и предобработаем для получения дневных данных
    day = soup.find_all("div", class_="border")[1]
    for i in day:
        day = list(i.find_all("div"))

    day = day[4:]

    day = [str(i) for i in day]

    day = [i.replace("<div>", "").replace("</div>", "") for i in day]

    day[0] = pd.to_datetime(day[0])

    day[1] = day[1].split("/")
    day[1] = day[1][1].strip().replace(",", "")
    day[1] = pd.to_numeric(day[1])

    day[2] = pd.to_numeric(day[2])

    day[3] = day[3].split("/")
    day[3] = day[3][1].strip().replace(",", "")
    day[3] = pd.to_numeric(day[3])

    day[4] = day[4].split("/")
    day[4] = day[4][1].strip().replace(",", "")
    day[4] = pd.to_numeric(day[4])

    df_row = (
        pd.DataFrame([day], columns=["Date", "Gold", "Silver", "Platinum", "Palladium"])
        .sort_values("Date")
        .reset_index(drop=True)
    )

    # Выделим данные и предобработаем для получения исторических данных
    historical = soup.find_all("div", class_="border")[2]
    for i in historical:
        historical = list(i.find_all("div"))

    historical = historical[5:]

    historical = [str(i) for i in historical]

    historical = [i.replace("<div>", "").replace("</div>", "") for i in historical]
    historical = [item for item in historical if "<div" not in item]

    # Разобьем по дням
    historical = [historical[i : i + 5] for i in range(0, len(historical), 5)]

    # Обработаем данные внутри каждого дня
    for i in historical:
        i[0] = pd.to_datetime(i[0])

        i[1] = i[1].split("/")
        i[1] = i[1][1].strip().replace(",", "")
        i[1] = pd.to_numeric(i[1])

        i[2] = pd.to_numeric(i[2])

        i[3] = i[3].split("/")
        i[3] = i[3][1].strip().replace(",", "")
        i[3] = pd.to_numeric(i[3])

        i[4] = i[4].split("/")
        i[4] = i[4][1].strip().replace(",", "")
        i[4] = pd.to_numeric(i[4])

    df_historical = (
        pd.DataFrame(
            historical, columns=["Date", "Gold", "Silver", "Platinum", "Palladium"]
        )
        .sort_values("Date")
        .reset_index(drop=True)
    )

    df_historical = pd.concat([df_historical, df_row]).reset_index(drop=True)
    kitko_db = pd.read_excel("./data/kitko_db.xlsx", index_col=0)

    kitko_db = (
        pd.concat([kitko_db, df_historical])
        .reset_index(drop=True)
        .drop_duplicates(subset=["Date"])
    )

    with pd.ExcelWriter(
        "./data/kitko_db.xlsx",
        date_format="YYYY-MM-DD",
        datetime_format="YYYY-MM-DD",
    ) as writer:
        kitko_db.to_excel(writer, sheet_name="kitco_metall")
    
    print("KITKO_main is done!!!") 
    
    return kitko_db

In [222]:
# Ассинхронная функция
async def kitko_parser():
    def kitco_raw():
        try:
                url = "https://www.kitco.com/price/fixes/london-fix"

                service = Service()
                driver = webdriver.Chrome(service=service)

                # driver.maximize_window()
                driver.get(url=url)

                time.sleep(2)

                driver.execute_script(
                    "window.scrollTo(0, window.scrollY + window.innerHeight);"
                )
                driver.execute_script(
                    "window.scrollTo(0, window.scrollY + window.innerHeight);"
                )

                time.sleep(2)

                html_code = driver.page_source

                return html_code

        except Exception as error:
            print(error)

        finally:
            driver.close()
            driver.quit()
    
    # Работа над полученными данными
    html_code = kitco_raw()
    soup = BeautifulSoup(html_code)
    
    # Выделим данные и предобработаем для получения дневных данных
    day = soup.find_all("div", class_="border")[1]
    for i in day:
        day = list(i.find_all("div"))

    day = day[4:]

    day = [str(i) for i in day]

    day = [i.replace("<div>", "").replace("</div>", "") for i in day]

    day[0] = pd.to_datetime(day[0])

    day[1] = day[1].split("/")
    day[1] = day[1][1].strip().replace(",", "")
    day[1] = pd.to_numeric(day[1])

    day[2] = pd.to_numeric(day[2])

    day[3] = day[3].split("/")
    day[3] = day[3][1].strip().replace(",", "")
    day[3] = pd.to_numeric(day[3])

    day[4] = day[4].split("/")
    day[4] = day[4][1].strip().replace(",", "")
    day[4] = pd.to_numeric(day[4])

    df_row = (
        pd.DataFrame([day], columns=["Date", "Gold", "Silver", "Platinum", "Palladium"])
        .sort_values("Date")
        .reset_index(drop=True)
    )

    # Выделим данные и предобработаем для получения исторических данных
    historical = soup.find_all("div", class_="border")[2]
    for i in historical:
        historical = list(i.find_all("div"))

    historical = historical[5:]

    historical = [str(i) for i in historical]

    historical = [i.replace("<div>", "").replace("</div>", "") for i in historical]
    historical = [item for item in historical if "<div" not in item]

    # Разобьем по дням
    historical = [historical[i : i + 5] for i in range(0, len(historical), 5)]

    # Обработаем данные внутри каждого дня
    for i in historical:
        i[0] = pd.to_datetime(i[0])

        i[1] = i[1].split("/")
        i[1] = i[1][1].strip().replace(",", "")
        i[1] = pd.to_numeric(i[1])

        i[2] = pd.to_numeric(i[2])

        i[3] = i[3].split("/")
        i[3] = i[3][1].strip().replace(",", "")
        i[3] = pd.to_numeric(i[3])

        i[4] = i[4].split("/")
        i[4] = i[4][1].strip().replace(",", "")
        i[4] = pd.to_numeric(i[4])

    df_historical = (
        pd.DataFrame(
            historical, columns=["Date", "Gold", "Silver", "Platinum", "Palladium"]
        )
        .sort_values("Date")
        .reset_index(drop=True)
    )

    df_historical = pd.concat([df_historical, df_row]).reset_index(drop=True)
    kitko_db = pd.read_excel("./data/kitko_db.xlsx", index_col=0)

    kitko_db = (
        pd.concat([kitko_db, df_historical])
        .reset_index(drop=True)
        .drop_duplicates(subset=["Date"])
    )

    with pd.ExcelWriter(
        "./data/kitko_db.xlsx",
        date_format="YYYY-MM-DD",
        datetime_format="YYYY-MM-DD",
    ) as writer:
        kitko_db.to_excel(writer, sheet_name="kitco_metall")
    
    print("KITKO_main is done!!!") 
    
    return kitko_db

await kitko_parser()

KITKO_main is done!!!


,Date,Gold,Silver,Platinum,Palladium
0,2023-01-03,1843.25,24.295,1082.0,1795.0
1,2023-01-04,1857.30,24.290,1080.0,1736.0
2,2023-01-05,1834.00,23.410,1062.0,1783.0
3,2023-01-06,1852.20,23.455,1073.0,1784.0
4,2023-01-09,1878.85,23.850,1092.0,1793.0
...,...,...,...,...,...
313,2024-04-02,2264.50,25.650,925.0,1016.0
314,2024-04-03,2280.15,26.250,929.0,1011.0
315,2024-04-04,2293.50,27.000,935.0,1024.0
316,2024-04-05,2298.55,26.780,923.0,998.0
